# Welcome To The Galapagos Setup

The Galapagos Middleware presents an orchestration framework to create multi-FPGA and CPU networks and can be built on top of any device that has a Galapagos Hypervisor.

<a id='own_example'></a>

# Setup

## Setup Environment Variables

The following cells follow the initial setup. The first sets up the environment variables needed by the scripts below.

In [ ]:
%%bash

galapagos_path=/home/user/galapagos
cd $galapagos_path

vivado_version=2018.1
hls_version=2018.1
galapagos_board=adm-8k5-debug
vivado_path=/opt/Xilinx/Vivado
hls_path=/opt/Xilinx/Vivado

source init.sh $galapagos_path $vivado_path $hls_path $vivado_version $hls_version
galapagos-update-board $galapagos_board

## Make Middleware IP


Next we need to build the IP cores used by the middleware layer to connect different Galapagos objects

In [ ]:
%%bash
source $GALAPAGOS_VIVADO_PATH/settings64.sh

cd $GALAPAGOS_PATH
make hlsmiddleware  

## Creating User Kernels 

To create a user kernel it is very similar to that of creating the hlsmiddleware in the previous section.
In the following we have an example.

First we set the board we wish to target:

In [ ]:
%%bash

galapagos-update-board adm-8k5-debug

Next we make the kernel:

In [ ]:
%%bash 

cd /home/user/HCAL_HLS4ML
make hls

This will place the generated IP cores in `<galapagos_home>/hlsBuild/<board_name>/ip`. Here we can see all the IP cores that we have created for this board. This includes the middleware IP cores and the User IP cores

In [ ]:
%%bash

cd $GALAPAGOS_PATH/hlsBuild/$GALAPAGOS_BOARD_NAME/ip
ls

Now that we have created our IP cores, we can describe them in a cluster and stitch them together. <a href='http://127.0.0.1:9000/notebooks/02_middleware.ipynb'>Using the Middleware'</a>